In [2]:
# ex12_1_3
import numpy as np
# Load data. There is probably a library-way to parse the file but we will take the scenic route
with open('./data/courses.txt','r') as f:
    D = f.read()
    print("Raw data matrix is:")
    print(D)
D = [ [int(x) for x in ds.split(",")] for ds in D.split('\n') if len(ds) > 0]
N = len(D)
M = max( [max(v) for v in D])
X = np.zeros( (N,M))
for i in range(N):
    d_m1 = [j-1 for j in D[i]]
    X[i,d_m1] = 1

# We should now have the correct binary data matrix:
labels = ["02322", "02450", "02451", "02453", "02454", "02457", "02459", "02582"]
print("Transformed data matrix X is:")
print(labels)
print(X)
print("All-done!")

Raw data matrix is:
2,5,6,7,8
1,2,3,6,7,8
2,4,6,8
3,6,7
2,6,7
2,3,6,7,8

Transformed data matrix X is:
['02322', '02450', '02451', '02453', '02454', '02457', '02459', '02582']
[[0. 1. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 0. 0. 1. 1. 1.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 0. 1. 1. 1.]]
All-done!


In [3]:
# install apyori using a standard method: conda install apyori or pip install apyori
from apyori import apriori
# Load resources from previous exercise
import numpy as np
#from ex12_1_3 import X,labels
# ex12_1_4
# This is a helper function that transforms a binary matrix into transactions.
# Note the format used for courses.txt was (nearly) in a transaction format,
# however we will need the function later which is why we first transformed
# courses.txt to our standard binary-matrix format.
def mat2transactions(X, labels=[]):
    T = []
    for i in range(X.shape[0]):
        l = np.nonzero(X[i, :])[0].tolist()
        if labels:
            l = [labels[i] for i in l]
        T.append(l)
    return T

# apyori requires data to be in a transactions format, forunately we just wrote a helper function to do that.
T = mat2transactions(X,labels)
rules = apriori( T, min_support=0.8, min_confidence=1)

# This function print the found rules and also returns a list of rules in the format:
# [(x,y), ...]
# where x -> y
def print_apriori_rules(rules):
    frules = []
    for r in rules:
        for o in r.ordered_statistics:        
            conf = o.confidence
            supp = r.support
            x = ", ".join( list( o.items_base ) )
            y = ", ".join( list( o.items_add ) )
            print("{%s} -> {%s}  (supp: %.3f, conf: %.3f)"%(x,y, supp, conf))
            frules.append( (x,y) )
    return frules
# Print rules found in the courses file.
print_apriori_rules(rules)

{} -> {02457}  (supp: 1.000, conf: 1.000)
{02450} -> {02457}  (supp: 0.833, conf: 1.000)
{02459} -> {02457}  (supp: 0.833, conf: 1.000)


[('', '02457'), ('02450', '02457'), ('02459', '02457')]

In [4]:
# ex12_1_5
# Load data from the wine dataset
from scipy.io import loadmat
mat_data = loadmat('./data/wine.mat')
X = mat_data['X']
y = mat_data['y'].squeeze()
attributeNames = [name[0][0] for name in mat_data['attributeNames']]

# We will now transform the wine dataset into a binary format. Notice the changed attribute names:
from similarity import binarize2
Xbin, attributeNamesBin = binarize2(X, attributeNames)
print("X, i.e. the wine dataset, has now been transformed into:")
print(Xbin)
print(attributeNamesBin)

X, i.e. the wine dataset, has now been transformed into:
[[1. 1. 0. ... 0. 1. 1.]
 [1. 1. 0. ... 0. 1. 1.]
 [1. 1. 0. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 1. 0. 1.]]
['Fixed acidity 50th-100th percentile', 'Volatile acidity 50th-100th percentile', 'Citric acid 50th-100th percentile', 'Residual sugar 50th-100th percentile', 'Chlorides 50th-100th percentile', 'Free sulfur dioxide 50th-100th percentile', 'Total sulfur dioxide 50th-100th percentile', 'Density 50th-100th percentile', 'pH 50th-100th percentile', 'Sulphates 50th-100th percentile', 'Alcohol 50th-100th percentile', 'Quality score (0--10) 50th-100th percentile', 'Fixed acidity 0th-50th percentile', 'Volatile acidity 0th-50th percentile', 'Citric acid 0th-50th percentile', 'Residual sugar 0th-50th percentile', 'Chlorides 0th-50th percentile', 'Free sulfur dioxide 0th-50th percentile', 'Total sulfur dioxide 0th-50th percentile', 'Density 0th-50th percentile', 'pH 0th-50th percentile',

In [5]:
# ex12_1_6
# Load resources from previous exercise
#from ex12_1_4 import mat2transactions, print_apriori_rules
#from ex12_1_5 import Xbin, attributeNamesBin
#from apyori import apriori

# Given the processed data in the previous exercise this becomes easy:
T = mat2transactions(Xbin,labels=attributeNamesBin)
rules = apriori(T, min_support=0.3, min_confidence=.6)
print_apriori_rules(rules)

{} -> {Quality score (0--10) 0th-50th percentile}  (supp: 0.803, conf: 0.803)
{Alcohol 0th-50th percentile} -> {Chlorides 50th-100th percentile}  (supp: 0.329, conf: 0.651)
{Chlorides 50th-100th percentile} -> {Alcohol 0th-50th percentile}  (supp: 0.329, conf: 0.667)
{Alcohol 0th-50th percentile} -> {Density 50th-100th percentile}  (supp: 0.381, conf: 0.755)
{Density 50th-100th percentile} -> {Alcohol 0th-50th percentile}  (supp: 0.381, conf: 0.764)
{Alcohol 0th-50th percentile} -> {Quality score (0--10) 0th-50th percentile}  (supp: 0.471, conf: 0.932)
{Alcohol 0th-50th percentile} -> {Total sulfur dioxide 50th-100th percentile}  (supp: 0.305, conf: 0.604)
{Total sulfur dioxide 50th-100th percentile} -> {Alcohol 0th-50th percentile}  (supp: 0.305, conf: 0.613)
{Alcohol 50th-100th percentile} -> {Chlorides 0th-50th percentile}  (supp: 0.330, conf: 0.668)
{Chlorides 0th-50th percentile} -> {Alcohol 50th-100th percentile}  (supp: 0.330, conf: 0.652)
{Alcohol 50th-100th percentile} -> {Den

[('', 'Quality score (0--10) 0th-50th percentile'),
 ('Alcohol 0th-50th percentile', 'Chlorides 50th-100th percentile'),
 ('Chlorides 50th-100th percentile', 'Alcohol 0th-50th percentile'),
 ('Alcohol 0th-50th percentile', 'Density 50th-100th percentile'),
 ('Density 50th-100th percentile', 'Alcohol 0th-50th percentile'),
 ('Alcohol 0th-50th percentile', 'Quality score (0--10) 0th-50th percentile'),
 ('Alcohol 0th-50th percentile', 'Total sulfur dioxide 50th-100th percentile'),
 ('Total sulfur dioxide 50th-100th percentile', 'Alcohol 0th-50th percentile'),
 ('Alcohol 50th-100th percentile', 'Chlorides 0th-50th percentile'),
 ('Chlorides 0th-50th percentile', 'Alcohol 50th-100th percentile'),
 ('Alcohol 50th-100th percentile', 'Density 0th-50th percentile'),
 ('Density 0th-50th percentile', 'Alcohol 50th-100th percentile'),
 ('Alcohol 50th-100th percentile',
  'Quality score (0--10) 0th-50th percentile'),
 ('Alcohol 50th-100th percentile', 'Total sulfur dioxide 0th-50th percentile'),
 (

In [6]:
X

array([[1., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 0., 1., 0.]])